# Homework 1: Preprocessing and Text Classification

Student Name: Shi Chang Zhang

Student ID: 695434

Python version used: 2.7

## General info

<b>Due date</b>: 5pm, Thursday March 16

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be using a corpus of tweets to do tokenisation of hashtags and build polarity classifers using bag of word (BOW) features.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gemsim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Preprocessing

<b>Instructions</b>: For this homework we will be using the tweets in the <i>twitter_samples</i> corpus included with NLTK. You should start by accessing these tweets. Use the <i>strings</i> method included in the NLTK corpus reader for <i>twitter_samples</i> to access the tweets (as raw strings). Iterate over the full corpus, and print out the average length, in characters, of the tweets in the corpus. (0.5)


In [117]:
from nltk.corpus import twitter_samples

tweets = twitter_samples.strings()

print(sum([len(tweet) for tweet in tweets]) / float(len(tweets)))

103.887266667


<b>Instructions</b>: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. You should use a regular expression to extract all hashtags of length 8 or longer which consist only of lower case letters (other than the # at the beginning, of course, though this should be stripped off as part of the extraction process). Do <b>not</b> tokenise the entire tweet as part of this process. The hashtag might occur at the beginning or the end of the tweet; you should double-check that you aren't missing any. After you have collected them into a list, print out number of hashtags you have collected: for full credit, you must get the exact number that we expect.  (1.0)

In [118]:
import re

hashtags = []

for tweet in tweets:
    hashtags += re.findall(r'(?:\A|\s)#([a-z]{8,})(?:\Z|\s)', tweet)
    
print(len(hashtags))

1238


<b>Instructions</b>: Now, tokenise the hashtags you've collected. To do this, you should implement a reversed version of the MaxMatch algorithm discussed in class (and in the reading), where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatiser before matching. Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenised hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list. (1.0)

In [119]:
import nltk

words = nltk.corpus.words.words() # words is a Python list

# Sorted for binary search
words = [word.lower() for word in words]
words.sort()

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

def binsearchin(value, dict):
    lower = 0
    upper = len(dict) - 1
    
    while lower <= upper:
        mid = (lower + upper) // 2
        x = dict[mid]
        if x < value:
            lower = mid + 1
        elif value < x:
            upper = mid - 1
        else:
            return True
    return False

def rmaxmatch(string, dict):
    if not string:
        return []
    for i in range(0, len(string)):
        lastword = string[i:]
        remainder = string[:i]
        if binsearchin(lemmatize(lastword), dict):
            return rmaxmatch(remainder, dict) + [lastword]
            
    lastword  = string[-1]
    remainder = string[:-1]
    return rmaxmatch(remainder, dict) + [lastword]

hashtagtokens = [rmaxmatch(hashtag, words) for hashtag in hashtags]

print(hashtagtokens[-20:])

[[u'scot', u'night'], [u'democrats'], [u'worrying'], [u'falling', u'labour'], [u'leaders', u'debate'], [u'wow', u'campaign'], [u'tory', u'lies'], [u'election'], [u'biased', u'b', u'b', u'c'], [u'labour', u'doorstep'], [u'biased', u'b', u'b', u'c'], [u'b', u'b', u'c', u'debate'], [u'mi', u'li', u'fandom'], [u'u', u'k', u'parliament'], [u'bedroom', u'tax'], [u'cannabis'], [u'vote', u'green'], [u'l', u'lan', u'el', u'li', u'hu', u'stings'], [u'bedroom', u'tax'], [u'bankrupt']]


### Extra Credit (Optional)
<b>Instructions</b>: Implement the forward version of the MaxMatch algorithm as well, and print out all the hashtags which give different results for the two versions of MaxMatch. Your main task is to come up with a good way to select which of the two segmentations is better for any given case, and demonstrate that it works significantly better than using a single version of the algorithm for all hashtags. (1.0)

In [120]:
def maxmatch(string, dict):
    if not string:
        return []
    for i in range(len(string), 0, -1):
        firstword = string[:i]
        remainder = string[i:]
        if binsearchin(lemmatize(firstword), dict):
            return [firstword] + maxmatch(remainder, dict)
            
    firstword  = string[0]
    remainder = string[1:]
    return [firstword] + rmaxmatch(remainder, dict)

def matchscore(string):
    score = 0
    if len(string) == 1:
        return 1
    for tweet in tweets:
        score += len(re.findall('\W'+string+'\W',tweet))
    return score

def bettermatch(string, dict):
    forward = maxmatch(string, dict)
    reverse = rmaxmatch(string, dict)
    
    fscore = sum([matchscore(s) for s in forward])
    rscore = sum([matchscore(s) for s in reverse])
    
    if fscore > rscore:
        return forward
    
    return reverse
test = 'whereisthesun'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))

test = 'bestoftheday'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))

test = 'imintoher'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))

test = 'horribleman'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))

test = 'voteukip'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))

test = 'idolfansave'
print (maxmatch(test, words))
print (rmaxmatch(test, words))
print (bettermatch(test, words))


['where', 'ist', 'hes', 'un']
['w', 'he', 'reis', 'the', 'sun']
['w', 'he', 'reis', 'the', 'sun']
['best', 'oft', 'he', 'day']
['be', 'stof', 'the', 'day']
['be', 'stof', 'the', 'day']
['imi', 'n', 'toher']
['i', 'min', 'toher']
['i', 'min', 'toher']
['horrible', 'man']
['h', 'or', 'rib', 'leman']
['horrible', 'man']
['vote', 'u', 'kip']
['v', 'o', 't', 'eu', 'kip']
['vote', 'u', 'kip']
['idol', 'fans', 'ave']
['idol', 'fan', 'save']
['idol', 'fan', 'save']


## Text classification (Not Optional)

<b>Instructions</b>: The twitter_sample corpus has two subcorpora corresponding to positive and negative tweets. You can access already tokenised versions using the <i> tokenized </i> method, as given in the code sample below. Iterate through these two corpora and build training, development, and test sets for use with Scikit-learn. You should exclude stopwords (from the built-in NLTK list) and tokens with non-alphabetic characters (this is very important you do this because emoticons were used to build the corpus, if you don't remove them performance will be artificially high). You should randomly split each subcorpus, using 80% of the tweets for training, 10% for development, and 10% for testing; make sure you do this <b>before</b> combining the tweets from the positive/negative subcorpora, so that the sets are <i>stratified</i>, i.e. the exact ratio of positive and negative tweets is preserved across the three sets. (1.0)

In [121]:
from nltk.corpus import stopwords
from random import shuffle

stops = set(stopwords.words('english'))

positive_tweets = nltk.corpus.twitter_samples.tokenized("positive_tweets.json")
negative_tweets = nltk.corpus.twitter_samples.tokenized("negative_tweets.json")

def removestopwords(tokens, stops):
    r = [i for i in tokens if i.lower() not in stops and 
                                        not re.search(r'[^a-zA-Z]', i)]
    return r

positive_tweets = [removestopwords(tweet, stops) for tweet in positive_tweets]
negative_tweets = [removestopwords(tweet, stops) for tweet in negative_tweets]

shuffle(positive_tweets)
shuffle(negative_tweets)

p1 = int(len(positive_tweets)*0.8)
n1 = int(len(negative_tweets)*0.8)
p2 = p1 + int(len(positive_tweets)*0.1)
n2 = n1 + int(len(negative_tweets)*0.1)
             
training = positive_tweets[:p1] + negative_tweets[:n1]
development = positive_tweets[p1:p2] + negative_tweets[n1:n2]
testing = positive_tweets[p2:] + negative_tweets[n2:]

shuffle(training)
shuffle(development)
shuffle(testing)

<b>Instructions</b>: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do <b>not</b> use crossvalidation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output. (1.0)

In [122]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn import cross_validation 

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
    

def prepare_tweets_data(tweets, feature_extractor):
    feature_matrix = []
    classifications = []
    for tweet in tweets:
        feature_dict = feature_extractor(tweet)
        feature_matrix.append(feature_dict)
        if tweet in positive_tweets:
            classifications.append('positive')
        else:
            classifications.append('negative')
    
    vectorizer = DictVectorizer()
    dataset = vectorizer.fit_transform(feature_matrix)
    return dataset, classifications

def check_accuracy(model, predictions, classifications):
    print "\n"+model+" accuracy"
    print accuracy_score(classifications,predictions)

dataset, classifications = prepare_tweets_data(development, get_BOW)
n_to_test = range(5,25)
clfs = [MultinomialNB(n, True, [0.52, 0.5]) for n in n_to_test]
for clf in clfs:
    predictions = cross_validation.cross_val_predict(clf, dataset, classifications, cv=10)
    check_accuracy("naive bayes", predictions, classifications)
    print(clf.get_params())

n_to_test = range(1,10)
clfs = [LogisticRegression(C=n/float(10)) for n in n_to_test]
for clf in clfs:
    predictions = cross_validation.cross_val_predict(clf, dataset, classifications, cv=10)
    check_accuracy("logistic regression", predictions, classifications)
    print(clf.get_params())


naive bayes accuracy
0.682
{'alpha': 5, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.68
{'alpha': 6, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.679
{'alpha': 7, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.68
{'alpha': 8, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.679
{'alpha': 9, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.679
{'alpha': 10, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.682
{'alpha': 11, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.683
{'alpha': 12, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.682
{'alpha': 13, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.683
{'alpha': 14, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.682
{'alpha': 15, 'fit_prior': True, 'class_prior': [0.52, 0.5]}

naive bayes accuracy
0.682


<b>Instructions</b>: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macroaveraged f-score for each classifier. Be sure to label your output. (0.5)

In [124]:
from sklearn.metrics import classification_report

def check_results(predictions, classifications):
    print "accuracy"
    print accuracy_score(classifications,predictions)
    print classification_report(classifications,predictions)

NBclf = MultinomialNB(12, True, [0.52, 0.5]) 
LRclf = LogisticRegression(C=0.8)

training_X, training_Y = prepare_tweets_data(training, get_BOW)
testing_X, testing_Y = prepare_tweets_data(training, get_BOW)

NBclf.fit(training_X, training_Y)
LRclf.fit(training_X, training_Y)

print("\nNaive Bayes\n")
predictions = NBclf.predict(testing_X)
check_results(predictions, testing_Y)


print("\nLogisticRegression\n")
predictions = LRclf.predict(testing_X)
check_results(predictions, testing_Y)


Naive Bayes

accuracy
0.82625
             precision    recall  f1-score   support

   negative       0.80      0.85      0.83      3861
   positive       0.85      0.80      0.83      4139

avg / total       0.83      0.83      0.83      8000


LogisticRegression

accuracy
0.925
             precision    recall  f1-score   support

   negative       0.92      0.92      0.92      3861
   positive       0.93      0.93      0.93      4139

avg / total       0.92      0.93      0.92      8000

